In [1]:
import pickle
import spacy
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.neural_network import MLPRegressor as mlp
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install rouge
from rouge import Rouge

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# change to path to dataset
file_name = "/home/priyank/Downloads/Summarization_Pickled_Data-20200506T174927Z-001/Summarization_Pickled_Data/cnn_dataset_1000_labelled.pkl"
stories = pickle.load(open(file_name, 'rb'))

In [0]:
# displaying the first datapoint
# verify correctness of load

# Uncomment to Display the First Datapoint
# print(stories[0])

In [0]:
# Required Models for glove
# in case of errors with conda, use this:
# conda install -c conda-forge spacy
# this is what worked for me :P

# !python -m spacy download en
# !python -m spacy download en_core_web_lg
!python -m spacy link en_core_web_lg en --force

# use the large model as the default model for English textual data

✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
# Initializing the processor
embedder = spacy.load('en')

In [0]:
# basic embeddings using averaged glove vectors
# using Spacy's large language model
def get_embedding(text):
    extract = embedder(text)
    total_sum = np.zeros(300)
    count = 0
    for token in extract:
        count += 1
        total_sum += np.asarray(token.vector)
    return total_sum / count

In [4]:
# bert embeddings using pretrained bert

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from transformers import GPT2Tokenizer
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', max_len = 1600)
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def get_embedding_bert_sent(text):
  marked_text = "[CLS] " + text + " [SEP]"

  # # Tokenize our sentence with the BERT tokenizer.
  tokenized_text = tokenizer.tokenize(marked_text)

  # # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  #Mark tokens according to the sentences they belong to
  segments_ids = [1] * len(tokenized_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Load pre-trained model (weights)
  model = BertModel.from_pretrained('bert-base-uncased')

  # Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()

  # Predict hidden states features for each layer
  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor, segments_tensors)

  # Concatenate the tensors for all layers. We use `stack` here to
  # create a new dimension in the tensor.
  token_embeddings = torch.stack(encoded_layers, dim=0)

  # Remove dimension 1, the "batches".
  token_embeddings = torch.squeeze(token_embeddings, dim=1)

  # Swap dimensions 0 and 1.
  token_embeddings = token_embeddings.permute(1,0,2)

  # `encoded_layers` has shape [12 x 1 x 22 x 768]

  # `token_vecs` is a tensor with shape [22 x 768]
  token_vecs = encoded_layers[11][0]

  # Calculate the average of all 22 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)

  return sentence_embedding.numpy()

  # print ("Our final sentence embedding vector of shape:", sentence_embedding.size())
  # print(sentence_embedding)

def get_embedding_bert_doc(list_doc):
  #text = sentence
  flag_token_id = 1
  segments_ids = []
  indexed_tokens = []
  for text in list_doc:
    marked_text = "[CLS] " + text + " [SEP]"

    # Tokenize our sentence with the BERT tokenizer.
    tokenized_text = tokenizer.tokenize(marked_text)

    # Map the token strings to their vocabulary indeces.
    for i in tokenizer.convert_tokens_to_ids(tokenized_text):
      indexed_tokens.append(i)

    #Mark tokens according to the sentences they belong to
    for i in range(len(tokenized_text)):
      segments_ids.append(flag_token_id)

    if flag_token_id == 1:
      flag_token_id = 0

    else:
      flag_token_id = 1

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Load pre-trained model (weights)
  model = BertModel.from_pretrained('bert-base-uncased')

  # Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()

  # Predict hidden states features for each layer
  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor, segments_tensors)

  # create a new dimension in the tensor.
  token_embeddings = torch.stack(encoded_layers, dim=0)
  # Remove dimension 1, the "batches".
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  # Swap dimensions 0 and 1.
  token_embeddings = token_embeddings.permute(1,0,2)
  token_vecs = encoded_layers[11][0]
  # Calculate the average of all 22 token vectors.
  document_embedding = torch.mean(token_vecs, dim=0)
  return document_embedding

In [ ]:
# creating the inputs and expected outputs
train_size = 100
val_size = 20
test_size = 20

def make_set(start_index, size):
    count = 0
    X_set = []
    y_set = []

    for count in tqdm(range(size)):
        data = stories[start_index + count]
        list_sent_emb = []
        for sent in data['story']:
              list_sent_emb.append(get_embedding_bert_sent(sent))

        doc_emb_list = np.array(list_sent_emb)
        doc_emb = np.mean(doc_emb_list, axis=0)
#         doc_emb = get_embedding(data['story_text'])
        # use the function of choice to generate the document embedding

        index = 0
        for sentence in data['story']:
#             sent_emb = get_embedding(sentence)
            sent_emb = get_embedding_bert_sent(sentence)
            # use the function of choice to generate the sentence embedding

            x = np.concatenate((sent_emb, doc_emb))
            try:
                y = data['scores'][index]
            except:
                y = 0.0
            index += 1

            X_set.append(x)
            y_set.append(y)

    return np.asmatrix(X_set), np.asarray(y_set)
print("done")
X_train, y_train = make_set(0, train_size)
print("done train")
X_val, y_val = make_set(train_size, val_size)
X_test, y_test = make_set(train_size + val_size, test_size)


  0%|          | 0/100 [00:00<?, ?it/s]

done


In [ ]:
def get_values(X, model):
    return model.predict(X)

def get_loss(pred, y):
    return np.linalg.norm(pred - y) / np.shape(y)[0]

model_name = "glove_averaged"
# modify the model name

def make_parameters(train_size):
    batch_size = int(4 * np.sqrt(train_size))
    n_batches = int(32 * (train_size / batch_size))
    # can set batch_size to standard values such as 64, 128, 256

    print("Total Number of Training Examples: " + str(train_size))
    print("Batch Size: " + str(batch_size))
    print("Number of Batches: " + str(n_batches))

    return batch_size, n_batches

def train(X_train, y_train, batch_size, n_batches):
    model = mlp(hidden_layer_sizes = (1024, 2048, 1024, 512, 256, 256, 128, 64), max_iter = 10000)
    
    train_size = np.shape(X_train)[0]

    min_loss = 1e20

    for iterator in tqdm(range(n_batches)):
        idx = np.random.randint(0, train_size, size = batch_size)

        X_select = X_train[idx,:]
        y_select = y_train[idx]

        model.partial_fit(X_select, y_select)

        sentence_predicted_scores = get_values(X_val, model)

        loss = get_loss(sentence_predicted_scores, y_val)

        # saving best model seen so far
        if loss < min_loss:
            min_loss = loss
            pickle.dump(model, open(model_name + '_best_model', 'wb'))

    final_model = pickle.load(open(model_name + '_best_model', 'rb'))
    return final_model

In [ ]:
batch_size, n_batches = make_parameters(train_size)

In [ ]:
m = train(X_train, 1000 * y_train, batch_size, n_batches)

In [ ]:
# Hyperparameter for similarity threshold
theta = 0.95

def similarity(A, B):
    similarity =  (A @ B.T) / (np.linalg.norm(A) * np.linalg.norm(B))
    return similarity

def get_top_k(X_doc, y, k):
    # k should be in {3, 4, 5}
    # error handling
    k = int(k)
    if k > 5:
        k = 5
    elif k < 3:
        k = 3
    
    order = np.flip(np.argsort(y))
    sentence_set = []
    for sent_id in order:
        if sentence_set == []:
            sentence_set.append(order[0])
            continue

        consider = X_doc[sent_id, :]
        flag = 1
        for consider_id in sentence_set:
            if similarity(X_doc[consider_id, :], consider) > theta:
                flag = 0
                break

        if flag == 1:
            sentence_set.append(sent_id)
    return sentence_set[0: min(k, len(sentence_set))]

In [ ]:
# Creating object of the ROUGE class
rouge = Rouge()

In [ ]:
# evaluation
# testing out each document iteratively
# test set: document 'train_size + val_size' onwards

def join(lst):
    string = ""
    for elem in lst:
        string = string + elem + " . "
    return string

def extract_rouge(rouge_dict):
    scores = []

    scores.append(100 * rouge_dict["rouge-1"]['f'])
    scores.append(100 * rouge_dict["rouge-1"]['p'])
    scores.append(100 * rouge_dict["rouge-1"]['r'])

    scores.append(100 * rouge_dict["rouge-2"]['f'])
    scores.append(100 * rouge_dict["rouge-2"]['p'])
    scores.append(100 * rouge_dict["rouge-2"]['r'])

    scores.append(100 * rouge_dict["rouge-l"]['f'])
    scores.append(100 * rouge_dict["rouge-l"]['p'])
    scores.append(100 * rouge_dict["rouge-l"]['r'])

    return np.asarray(scores)

start_doc_id = train_size + val_size
doc_count = len(stories)

generated_summary, gold_summary = 0, 0
# to access the final created summary

# set the number of documents for testing
limit = test_size

result = {}
result['3'] = np.zeros(9)
result['4'] = np.zeros(9)
result['5'] = np.zeros(9)
# averaging the ROUGE Metrics
# for different summary lengths

count = 0

while count < min(doc_count, limit):
    X_doc = []
    y_doc = []
    data = stories[start_doc_id + count]
    doc_emb = get_embedding(data['story_text'])

    index = 0
    for sentence in data['story']:
        sent_emb = get_embedding(sentence)

        x = np.concatenate((sent_emb, doc_emb))
        try:
            y = data['scores'][index]
        except:
            y = 0.0

        index += 1

        X_doc.append(x)
        y_doc.append(y)

    X_doc = np.asmatrix(X_doc)
    y_doc = np.asarray(y_doc)

    sentence_predicted_scores = get_values(X_doc, m)

    loss = np.linalg.norm(sentence_predicted_scores - y_doc)

    # Uncomment to view the test_loss on the sample  
    # print(loss)

    gold_summary = join(data['highlights'])

    for k in [3, 4, 5]:
        summary_sent_id = get_top_k(X_doc, sentence_predicted_scores, k)
        # Uncomment to view the indices of chosen sentences
        # print("Document ID:", start_doc_id + count, ", Top 5 Sentences:", summary_sent_id)

        # Uncomment to view the top 10 sentences based on Gold Labels
        # print("Top 10 sentences based on Gold Label", np.ndarray.tolist(np.flip(np.argsort(y_doc))[0:10]))

        generated_summary = join([data['story'][idx] for idx in summary_sent_id])

        scores = rouge.get_scores(generated_summary, gold_summary)[0]
        result[str(k)] += extract_rouge(scores)

    count += 1

for k in [3, 4, 5]:
    result[str(k)] = result[str(k)] / test_size

predicted = get_values(X_test, m)
test_loss = get_loss(y_test, predicted)

print("Sample Output:")
print("Document:\n", stories[-1]['story_text'])
print("Generated Summary:\n", generated_summary)
print("Gold Summary:\n", gold_summary)

print("\nAll Metrics:\n")

data = []
for k in [3, 4, 5]:
    lst = np.ndarray.tolist(result[str(k)])
    lst.append(test_loss)
    data.append(lst)

df = pd.DataFrame(data, columns = ['R1-f', 'R1-p', 'R1-r',
                                    'R2-f', 'R2-p', 'R2-r',
                                    'Rl-f', 'Rl-p', 'Rl-r',
                                    'Loss'], dtype = float)

df.index = ['glove top-3', 'glove top-4', 'glove top-5']
display(df)

# save results into a dataframe file
df.to_csv(model_name + '_results.csv')

In [0]:
# ^_^ Thank You